## Backend for Aspect-Based Sentiment Analysis & Forecasting of Customer Reviews

Kaggle submission for https://www.kaggle.com/competitions/bigquery-ai-hackathon

*Author: Michael Enudi*

For the write up including problem and impact statement, click [here](https://www.kaggle.com/competitions/bigquery-ai-hackathon/writeups/backend-for-aspect-based-sentiment-analysis-and-fo)

This notebook contain the implementation of the solution.

#### Data sourcing
The raw data was gotten extracted from Yelp Open Dataset and copied over to Cloud Storage (GCS) in the location `/yelp-sample-data-1/reviews`.

Once the data is in GCS, we load it into a `raw_review` table.

In [11]:
%%bigquery
LOAD DATA INTO yelp_analytics.raw_review
FROM FILES(
  format = 'NEWLINE_DELIMITED_JSON',
  uris = ['gs://yelp-sample-data-1/reviews/yelp_academic_dataset_review.json']
);

Query is running:   0%|          |

""


Because the raw data contains 6990280 records which is too large and would be costly for our prototype solution, we will create a smaller set of records called `small_review`.

In [4]:
%%bigquery
CREATE OR REPLACE TABLE `yelp_analytics.small_review` AS
SELECT review_id, user_id, business_id, text, stars, cool, funny, useful, date FROM `kaggle-comp-202509.yelp_analytics.raw_review`
WHERE date > '2022-01-01 23:59:59 UTC'

Query is running:   0%|          |

""


Let's see the number of records in `small_review`

In [5]:
%%bigquery
select count(1) from `kaggle-comp-202509`.yelp_analytics.small_review;

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,29997


### Create or replace our gemini model

Note: I created a connection using BigQuery UI with the connection ID `projects/kaggle-comp-202509/locations/us-east1/connections/gemini_conn`

In [6]:
%%bigquery
CREATE OR REPLACE MODEL `yelp_analytics.gemini_flash_lite`
REMOTE WITH CONNECTION `projects/kaggle-comp-202509/locations/us-east1/connections/gemini_conn`
OPTIONS (endpoint = 'gemini-2.5-flash-lite');

Query is running:   0%|          |

""


### Sentiment Analysis
Using our newly created gemini-2.5-flash-lite model, we will perform
- aspect extraction into some specific taxonomy categories
- sentiment analysis on each review
- sentiment analysis on each aspect

The results will be stored in a table name `aspect_extracted_review`

In [7]:
%%bigquery
CREATE OR REPLACE TABLE `yelp_analytics.aspect_extracted_review` AS
WITH model_result AS
  (
    SELECT
      review_id,
      REPLACE(
        REPLACE(JSON_VALUE(ml_generate_text_result, "$.candidates[0].content.parts[0].text"), '```json\n', ''), '\n```', ''
      ) AS json_payload
    FROM
      ML.GENERATE_TEXT(
        MODEL `yelp_analytics.gemini_flash_lite`,
        (
          SELECT
            review_id,
            'You are an AI assistant that performs aspect-based sentiment analysis on yelp reviews. Perform the following on the given review:' ||
            '1. Identify all relevant aspects or keywords mentioned in the text. ' ||
            '2. Assign a sentiment score to each aspect: Positive=1, Neutal=0, Negative=-1. Return the number only. ' ||
            '3. Classify each aspect into one of the following taxonomy categories: Food & Drinks, Service & Staff, Price & Value, Ambiance & Environment, Cleanliness & Hygiene, Others. ' ||
            '5. Assign a sentiment and calculate a sentiment score (between -1.0 to 1.0) for the entire body of text. ' ||
            '6. Decode the language for in which the review was written. ' ||
            '7. Return the sentiment (Positive=1, Neutal=0, Negative=-1: number only), sentiment_score, language and the aspects in JSON format, where the aspects is a list of objects as follows: aspect, sentiment, category. ' ||
            'Review text is: ' || text AS prompt
          FROM
            `yelp_analytics.small_review`
        ),
        STRUCT(
         4096 AS max_output_tokens
        )
      ) AS analysis
  )
  SELECT
  review_id,
  json_payload AS text,
  JSON_VALUE(json_payload, "$.sentiment") AS sentiment,
  JSON_VALUE(json_payload, "$.sentiment") AS sentiment_score,
  JSON_VALUE(json_payload, "$.language") AS language,
  JSON_QUERY(json_payload, "$.aspects") AS aspects
FROM model_result;

Query is running:   0%|          |

""


## Milestone 1

Join the `aspect_extracted_review` result with the other details into a new table called `small_review_processed`.

This is the first main table of our little workflow.
From this table, many analysis task can be performed along different hierarchy od taxonomy and/or date dimensions.

In [14]:
%%bigquery
CREATE OR REPLACE TABLE `yelp_analytics.small_review_processed` AS
SELECT
  t1.review_id,
  t1.user_id,
  t1.business_id,
  EXTRACT(YEAR FROM t1.date) AS review_year,
  EXTRACT(MONTH FROM t1.date) AS review_month,
  EXTRACT(DAY FROM t1.date) AS review_day,
  EXTRACT(DAYOFWEEK FROM t1.date) AS review_day_of_week,
  EXTRACT(HOUR FROM t1.date) AS review_hour_of_day,
  t1.date as review_timestamp,
  t1.stars,
  t1.cool,
  t1.funny,
  t1.useful,
  CAST(t2.sentiment AS numeric) AS sentiment,
  CAST(t2.sentiment_score AS numeric) AS sentiment_score,
  t2.language,
  t2.aspects
FROM
  `yelp_analytics.small_review` AS t1
INNER JOIN
  `yelp_analytics.aspect_extracted_review` AS t2 ON t1.review_id = t2.review_id;

Query is running:   0%|          |

""


In [15]:
%%bigquery
CREATE OR REPLACE TABLE `yelp_analytics.weekly_review_aspect_sentiment_aggregate` AS
SELECT
  t2.category,
  t2.aspect,
  DATE_TRUNC(t1.review_timestamp, WEEK) AS week_start,
  COUNT(1) AS total_mentions,
  COUNTIF(t2.aspect_sentiment = 1) AS positive_mentions,
  COUNTIF(t2.aspect_sentiment = -1) AS negative_mentions,
  COUNTIF(t2.aspect_sentiment = 0) AS neutral_mentions,
  MIN(t1.sentiment_score) AS min_review_sentiment_score,
  MAX(t1.sentiment_score) AS max_review_sentiment_score,
  AVG(t1.sentiment_score) AS avg_review_sentiment_score
FROM
  `yelp_analytics.small_review_processed` AS t1,
  UNNEST(JSON_EXTRACT_ARRAY(t1.aspects)) AS aspect_data,
  UNNEST([STRUCT(
    JSON_EXTRACT_SCALAR(aspect_data, '$.aspect') AS aspect,
    CAST(JSON_EXTRACT_SCALAR(aspect_data, '$.sentiment') AS NUMERIC) AS aspect_sentiment,
    JSON_EXTRACT_SCALAR(aspect_data, '$.category') AS category
  )]) AS t2
GROUP BY
  week_start, t2.category, t2.aspect
ORDER BY
  week_start ASC,
  total_mentions DESC;


Query is running:   0%|          |

""


Let's see the distribution of categories that we have in our aggregation table.

In [18]:
%%bigquery
select category, count(1) from `yelp_analytics.weekly_review_aspect_sentiment_aggregate` group by category limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,category,f0_
0,Food & Drinks,44034
1,Service & Staff,28560
2,Price & Value,4472
3,Others,24586
4,Ambiance & Environment,10033
5,Cleanliness & Hygiene,2422
6,Service & Value,1
7,Service & Drinks,1
8,Service & Environment,1


## Forecast models
Create five different forecast models to train ARIMA_PLUS models on weekly average sentiment score across the various taxonomy categories.

In [26]:
%%bigquery
CREATE OR REPLACE MODEL yelp_analytics.weekly_avg_sentiment_forecast_food_drinks
OPTIONS(
  MODEL_TYPE = 'ARIMA_PLUS',
  time_series_timestamp_col = 'week_start',
  time_series_data_col = 'avg_review_sentiment_score',
  data_frequency = 'WEEKLY'
) AS
SELECT week_start, avg_review_sentiment_score FROM `yelp_analytics.weekly_review_aspect_sentiment_aggregate` WHERE category = 'Food & Drinks';


CREATE OR REPLACE MODEL yelp_analytics.weekly_avg_sentiment_forecast_staff_service
OPTIONS(
  MODEL_TYPE = 'ARIMA_PLUS',
  time_series_timestamp_col = 'week_start',
  time_series_data_col = 'avg_review_sentiment_score',
  data_frequency = 'WEEKLY'
) AS
SELECT week_start, avg_review_sentiment_score FROM `yelp_analytics.weekly_review_aspect_sentiment_aggregate` WHERE category = 'Service & Staff';


CREATE OR REPLACE MODEL yelp_analytics.weekly_avg_sentiment_forecast_price_value
OPTIONS(
  MODEL_TYPE = 'ARIMA_PLUS',
  time_series_timestamp_col = 'week_start',
  time_series_data_col = 'avg_review_sentiment_score',
  data_frequency = 'WEEKLY'
) AS
SELECT week_start, avg_review_sentiment_score FROM `yelp_analytics.weekly_review_aspect_sentiment_aggregate` WHERE category = 'Price & Value';


CREATE OR REPLACE MODEL yelp_analytics.weekly_avg_sentiment_forecast_ambience_entertainment
OPTIONS(
  MODEL_TYPE = 'ARIMA_PLUS',
  time_series_timestamp_col = 'week_start',
  time_series_data_col = 'avg_review_sentiment_score',
  data_frequency = 'WEEKLY'
) AS
SELECT week_start, avg_review_sentiment_score FROM `yelp_analytics.weekly_review_aspect_sentiment_aggregate` WHERE category = 'Ambiance & Environment';


CREATE OR REPLACE MODEL yelp_analytics.weekly_avg_sentiment_forecast_clean_hygiene
OPTIONS(
  MODEL_TYPE = 'ARIMA_PLUS',
  time_series_timestamp_col = 'week_start',
  time_series_data_col = 'avg_review_sentiment_score',
  data_frequency = 'WEEKLY'
) AS
SELECT week_start, avg_review_sentiment_score FROM `yelp_analytics.weekly_review_aspect_sentiment_aggregate` WHERE category = 'Cleanliness & Hygiene';

Query is running:   0%|          |

""


## Milestone 2:
Generate 1 week forecast for sentiments for our specific categories

In [31]:
%%bigquery
SELECT
  'Ambiance & Environment' as category,
  forecast_timestamp as week_start,
  forecast_value,
  confidence_level,
  prediction_interval_lower_bound as lower_bound,
  prediction_interval_upper_bound as upper_bound
FROM ML.FORECAST(MODEL yelp_analytics.weekly_avg_sentiment_forecast_ambience_entertainment, STRUCT(1 AS horizon))
UNION ALL
SELECT
  'Cleanliness & Hygiene' as category,
  forecast_timestamp as week_start,
  forecast_value,
  confidence_level,
  prediction_interval_lower_bound as lower_bound,
  prediction_interval_upper_bound as upper_bound
FROM
  ML.FORECAST(MODEL yelp_analytics.weekly_avg_sentiment_forecast_clean_hygiene, STRUCT(1 AS horizon))
UNION ALL
SELECT
  'Food & Drinks' as category,
  forecast_timestamp as week_start,
  forecast_value,
  confidence_level,
  prediction_interval_lower_bound as lower_bound,
  prediction_interval_upper_bound as upper_bound
FROM
  ML.FORECAST(MODEL yelp_analytics.weekly_avg_sentiment_forecast_food_drinks, STRUCT(1 AS horizon))
UNION ALL
SELECT
  'Price & Value' as category,
  forecast_timestamp as week_start,
  forecast_value,
  confidence_level,
  prediction_interval_lower_bound as lower_bound,
  prediction_interval_upper_bound as upper_bound
FROM
  ML.FORECAST(MODEL yelp_analytics.weekly_avg_sentiment_forecast_price_value, STRUCT(1 AS horizon))
UNION ALL
SELECT
  'Service & Staff' as category,
  forecast_timestamp as week_start,
  confidence_level,
  forecast_value,
  prediction_interval_lower_bound as lower_bound,
  prediction_interval_upper_bound as upper_bound
FROM
  ML.FORECAST(MODEL yelp_analytics.weekly_avg_sentiment_forecast_staff_service, STRUCT(1 AS horizon))


Query is running:   0%|          |

Downloading:   0%|          |

,category,week_start,forecast_value,confidence_level,lower_bound,upper_bound
0,Food & Drinks,2022-01-23 00:00:00+00:00,0.525019,0.950000,0.525019,0.525019
1,Price & Value,2022-01-23 00:00:00+00:00,-0.168734,0.950000,-0.169239,-0.168228
2,Cleanliness & Hygiene,2022-01-23 00:00:00+00:00,-0.377126,0.950000,-0.377126,-0.377126
3,Ambiance & Environment,2022-01-23 00:00:00+00:00,0.538537,0.950000,0.538536,0.538538
4,Service & Staff,2022-01-23 00:00:00+00:00,0.950000,-0.004067,-0.009132,0.000998
